In [75]:
import numpy as np
import itertools

# 1.1 Реализовать функцию REF(), приводящую матрицу к ступенчатому виду.

In [76]:
def ref(matrix):
    matrix = np.array(matrix, dtype=int)

    rows, cols = matrix.shape
    lead = 0

    for r in range(rows):
        if lead >= cols:
            return matrix

        i = r
        while matrix[i, lead] == 0:
            i += 1
            if i == rows:
                i = r
                lead += 1
                if lead == cols:
                    return matrix

        matrix[[i, r]] = matrix[[r, i]]

        for i in range(r + 1, rows):
            if matrix[i, lead] == 1:
                matrix[i] = matrix[i] ^ matrix[r]

        lead += 1

    return matrix

In [77]:
m = [[1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1],
     [0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0],
     [0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1],
     [0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0],
     [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1]]
ref(m)

array([[1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1],
       [0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0],
       [0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1]])

# 1.2. Реализовать функцию RREF(), приводящую матрицу к приведённому ступенчатому виду.

In [78]:
def rref(matrix):
    matrix = np.array(matrix, dtype=float)
    rows, cols = matrix.shape
    lead = 0

    for r in range(rows):
        if lead >= cols:
            return matrix

        i = r
        while matrix[i, lead] == 0:
            i += 1
            if i == rows:
                i = r
                lead += 1
                if lead == cols:
                    return matrix

        matrix[[i, r]] = matrix[[r, i]]

        lv = matrix[r, lead]
        matrix[r] = matrix[r] / lv

        for i in range(rows):
            if i != r:
                lv = matrix[i, lead]
                matrix[i] = matrix[i] - lv * matrix[r]

        lead += 1

    return matrix

In [79]:
rref(m)

array([[ 1.,  0.,  1.,  0.,  0., -1.,  0.,  1.,  0., -1.,  2.],
       [ 0.,  0.,  0.,  1.,  0.,  1.,  0.,  0.,  0.,  1., -1.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,  0.,  1.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  1.]])

# 1.3. Создать класс линейных кодов LinearCode.

In [80]:
class LinearCode:
    def __init__(self, matrix):
        self.matrix = np.array(matrix, dtype=int)
        self.lead = []
        self.check = []

    def ref(self):
        rows, cols = self.matrix.shape
        lead = 0
        
        for r in range(rows):
            if lead >= cols:
                return
        
            i = r
            while self.matrix[i, lead] == 0:
                i += 1
                if i == rows:
                    i = r
                    lead += 1
                    if lead == cols:
                        return

            self.matrix[[i, r]] = self.matrix[[r, i]]
        
            for i in range(r + 1, rows):
                if self.matrix[i, lead] == 1:
                    self.matrix[i] = self.matrix[i] ^ self.matrix[r]
        
            lead += 1

    def rref(self):
        rows, cols = self.matrix.shape
        lead = 0
        
        for r in range(rows):
            if lead >= cols:
                return
        
            i = r
            while self.matrix[i, lead] == 0:
                i += 1
                if i == rows:
                    i = r
                    lead += 1
                    if lead == cols:
                        return

            self.matrix[[i, r]] = self.matrix[[r, i]]
        
            lv = self.matrix[r, lead]
            self.matrix[r] = self.matrix[r] / lv
        
            for i in range(rows):
                if i != r:
                    lv = self.matrix[i, lead]
                    self.matrix[i] = self.matrix[i] - lv * self.matrix[r]
        
            lead += 1

    def delete_null(self):
        for i in range(0,self.matrix.shape[0]):
            if 1 not in self.matrix[i,:]:
                self.matrix = np.delete(self.matrix, i, axis=0)

    def count_k_n(self):
        return self.matrix.shape[1], self.matrix.shape[0]

    def fix_lead(self):
        self.lead = []
        for i in range(0, self.matrix.shape[0]):
            ones = np.where(self.matrix[i, :] == 1)
            if ones[0].size == 0:
                return
            else:
                self.lead.append(ones[0][0])

    def delete_lead(self):
        for i in range(0, len(self.lead)):
            self.matrix = np.delete(self.matrix, self.lead[i] - i, axis=1)

    def checking_matrix(self):
        self.check = []
        temp = np.eye(self.matrix.shape[1], dtype = int)
        k = s = 0
        for i in range(0, self.matrix.shape[0] + self.matrix.shape[1]):
            if i in self.lead:
                self.check.append(self.matrix[k])
                k += 1
            else:
                self.check.append(temp[s])
                s += 1
        self.check = np.array(self.check)

    def generate_code_words_summing(self):
        """ Формируем кодовые слова, складывая все слова из порождающего множества """
        zeros = np.zeros(self.matrix.shape[1], dtype=int)
        final_word = set()
        final_word.add(tuple(zeros.tolist()))
        for i in range(1, self.matrix.shape[0] + 1):
            words = list(itertools.combinations(range(self.matrix.shape[0]), i))
            for c in words:
                word = np.zeros(self.matrix.shape[1], dtype=int)
                for j in c:
                    word += self.matrix[j, :]
                word %= 2
                final_word.add(tuple(word.tolist()))
        return np.array(list(final_word))

    def generate_code_words_k(self):
        """ Формируем кодовые слова, беря все кодовые слова длины k и умножая на матрицу """
        n, k = self.count_k_n()
        final_code = itertools.product(range(2), repeat=k)
        answer = []
        for code in final_code:
            one_word = np.dot(code,self.matrix) % 2
            answer.append(one_word)
        return answer

## 1.3.1 На основе входной матрицы сформировать порождающую матрицу в ступенчатом виде.

In [81]:
linear_code = LinearCode(m)
linear_code.rref()
linear_code.delete_null()
linear_code.matrix

array([[ 1,  0,  1,  0,  0, -1,  0,  1,  0, -1,  2],
       [ 0,  0,  0,  1,  0,  1,  0,  0,  0,  1, -1],
       [ 0,  0,  0,  0,  1,  0,  0,  1,  0,  0,  1],
       [ 0,  0,  0,  0,  0,  0,  1,  0,  0,  1,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  1,  1,  1]])

## 1.3.2 Задать n равное числу столбцов и k равное числу строк полученной матрицы (без учёта полностью нулевых строк).

In [82]:
linear_code.count_k_n()

(11, 5)

## Шаг 2. Зафиксировать ведущие столбцы lead матрицы 𝐆∗.

In [83]:
linear_code.fix_lead()
linear_code.lead

[np.int64(0), np.int64(3), np.int64(4), np.int64(6), np.int64(8)]

## Шаг 3. Сформировать сокращённую матрицу 𝐗, удалив ведущие столбцы матрицы 𝐆∗.

In [84]:
linear_code.delete_lead()
linear_code.matrix

array([[ 0,  1, -1,  1, -1,  2],
       [ 0,  0,  1,  0,  1, -1],
       [ 0,  0,  0,  1,  0,  1],
       [ 0,  0,  0,  0,  1,  0],
       [ 0,  0,  0,  0,  1,  1]])

## Шаг 4. Сформировать матрицу 𝐇, поместив в строки, соответствующие позициям ведущих столбцов строки из 𝐗, а в остальные – строки единичной матрицы.

In [85]:
linear_code.checking_matrix()
linear_code.check

array([[ 0,  1, -1,  1, -1,  2],
       [ 1,  0,  0,  0,  0,  0],
       [ 0,  1,  0,  0,  0,  0],
       [ 0,  0,  1,  0,  1, -1],
       [ 0,  0,  0,  1,  0,  1],
       [ 0,  0,  1,  0,  0,  0],
       [ 0,  0,  0,  0,  1,  0],
       [ 0,  0,  0,  1,  0,  0],
       [ 0,  0,  0,  0,  1,  1],
       [ 0,  0,  0,  0,  1,  0],
       [ 0,  0,  0,  0,  0,  1]])

# 1.4. Сформировать все кодовые слова длины n двумя способами.

In [86]:
G = np.array([
    [1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1],
    [0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0],
    [0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1],
    [0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1]
])

H = np.array([
    [0, 1, 1, 1, 1, 0],
    [1, 0, 0, 0, 0, 0],
    [0, 1, 0, 0, 0, 0],
    [0, 0, 1, 0, 1, 1],
    [0, 0, 0, 1, 0, 1],
    [0, 0, 1, 0, 0, 0],
    [0, 0, 0, 0, 1, 0],
    [0, 0, 0, 1, 0, 0],
    [0, 0, 0, 0, 1, 1],
    [0, 0, 0, 0, 1, 0],
    [0, 0, 0, 0, 0, 1]
])

linear_code = LinearCode(G)

## 1.4.1 Сложить все слова из порождающего множества, оставить неповторяющиеся.

In [87]:
code_words_summing = linear_code.generate_code_words_summing()
code_words_summing

array([[0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0],
       [0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1],
       [1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0],
       [1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0],
       [0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0],
       [0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0],
       [0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0],
       [1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1],
       [0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1],
       [1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0],
       [1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1],
       [0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1],
       [0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1],
       [1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1],
       [1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1],
       [1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0],
       [1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1],
       [0, 0, 0, 1, 0, 1, 1, 0, 0,

## 1.4.2 Взять все двоичные слова длины k, умножить каждое на G.

In [88]:
code_words_k = linear_code.generate_code_words_k()
code_words_k

[array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 array([0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1]),
 array([0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0]),
 array([0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1]),
 array([0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1]),
 array([0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0]),
 array([0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1]),
 array([0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0]),
 array([0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0]),
 array([0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1]),
 array([0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0]),
 array([0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1]),
 array([0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1]),
 array([0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0]),
 array([0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1]),
 array([0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0]),
 array([1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1]),
 array([1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0]),
 array([1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1]),
 array([1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0]),
 array([1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0]),
 array([1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1]),
 array([1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0]),
 array([1, 

### Убедиться, что полученные множества кодовых слов совпадают.

In [89]:
set_code_word_summing = set()
for word in code_words_summing:
    set_code_word_summing.add(tuple(word.tolist()))

set_code_word_k = set()
for word in code_words_k:
    set_code_word_k.add(tuple(word.tolist()))
    
set_code_word_summing == set_code_word_k

True

### Убедиться, что умножение кодовых слов на проверочную матрицу в результате даёт нулевые вектора

In [90]:
check_zero_matrix = []
for code in set_code_word_k:
    one_word = np.dot(code, H) % 2
    check_zero_matrix.append(one_word)
check_zero_matrix

[array([0, 0, 0, 0, 0, 0]),
 array([0, 0, 0, 0, 0, 0]),
 array([0, 0, 0, 0, 0, 0]),
 array([0, 0, 0, 0, 0, 0]),
 array([0, 0, 0, 0, 0, 0]),
 array([0, 0, 0, 0, 0, 0]),
 array([0, 0, 0, 0, 0, 0]),
 array([0, 0, 0, 0, 0, 0]),
 array([0, 0, 0, 0, 0, 0]),
 array([0, 0, 0, 0, 0, 0]),
 array([0, 0, 0, 0, 0, 0]),
 array([0, 0, 0, 0, 0, 0]),
 array([0, 0, 0, 0, 0, 0]),
 array([0, 0, 0, 0, 0, 0]),
 array([0, 0, 0, 0, 0, 0]),
 array([0, 0, 0, 0, 0, 0]),
 array([0, 0, 0, 0, 0, 0]),
 array([0, 0, 0, 0, 0, 0]),
 array([0, 0, 0, 0, 0, 0]),
 array([0, 0, 0, 0, 0, 0]),
 array([0, 0, 0, 0, 0, 0]),
 array([0, 0, 0, 0, 0, 0]),
 array([0, 0, 0, 0, 0, 0]),
 array([0, 0, 0, 0, 0, 0]),
 array([0, 0, 0, 0, 0, 0]),
 array([0, 0, 0, 0, 0, 0]),
 array([0, 0, 0, 0, 0, 0]),
 array([0, 0, 0, 0, 0, 0]),
 array([0, 0, 0, 0, 0, 0]),
 array([0, 0, 0, 0, 0, 0]),
 array([0, 0, 0, 0, 0, 0]),
 array([0, 0, 0, 0, 0, 0])]

# 1.5 Вычислить кодовое расстояние получившегося кода.

In [91]:
sums = np.sum(G, axis=1)
d = sums[np.argmin(sums)]
t = d - 1

d, t

(np.int64(2), np.int64(1))

## 1.4.1 Внести в кодовое слово ошибку кратности не более t, умножить полученное слово на H, убедиться в обнаружении ошибки.

In [92]:
v = np.array([1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0])
e1 = np.array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0])
s = (v + e1) % 2
print(f'v + e1 = {s}')

s2 = np.dot(s, H) % 2
print(f'(v + e1)@H = {s2} - error')

v + e1 = [1 0 0 1 1 0 1 0 0 1 0]
(v + e1)@H = [0 1 0 0 0 0] - error


## 1.4.2 Найти для некоторого кодового слова ошибку кратности t+1 такую, что при умножении на H ошибка не может быть обнаружена.

In [93]:
is_find = 0
print(v)

for i in range(len(v)):
    if is_find:
        break
    for j in range(len(v)):
        if i != j:
            e2 = np.zeros(len(v), dtype=int)
            e2[i], e2[j] = 1, 1

            s = (v + e2) % 2
            s2 = np.dot(s, H) % 2
            if sum(s2) == 0:
                is_find = 1
                break

print(f'e2 = {e2}')
print(f'v + e2 = {s}')
print(f'(v + e2)@H = {s2} - no error')

[1 0 1 1 1 0 1 0 0 1 0]
e2 = [0 0 0 0 0 0 1 0 0 1 0]
v + e2 = [1 0 1 1 1 0 0 0 0 0 0]
(v + e2)@H = [0 0 0 0 0 0] - no error
